# VacationPy
---

##  Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

City_ID             City      Lat       Lng  Max Temp  Humidity  \
0          0       whitehorse  60.7161 -135.0538     22.79        51   
1          1         yangambi   0.8102   24.4336     23.52        75   
2          2    haiku-pauwela  20.9219 -156.3051     29.01        65   
3          3     port lincoln -34.7333  135.8667     12.82        91   
4          4  roxborough park  39.4739 -105.0853     35.30        22   
..       ...              ...      ...       ...       ...       ...   
571      571            nadym  65.5333   72.5167     18.60        76   
572      572          kolwezi -10.7148   25.4667     15.55        43   
573      573       tselinnoye  53.0833   85.6667     18.12        72   
574      574         paracuru  -3.4100  -39.0306     26.73        83   
575      575          pachino  36.7186   15.0907     25.08        73   

     Cloudiness  Wind Speed Country        Date  
0            75        4.63      CA  1690493958  
1            55        0.60      CD  1690494413  
2            20       10.80      US  1690494282  
3            13        5.50      AU  1690494413  
4            93        2.24      US  1690494413  
..          ...         ...     ...         ...  
571          83        1.38      RU  1690494508  
572          58        2.18      CD  1690494512  
573          91        2.63      RU  1690494512  
574          12        6.80      BR  1690494512  
575           0        3.29      IT  1690494512  

[576 rows x 10 columns]

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map

map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_ideal=city_data_df[ (city_data_df['Max Temp']>20)&(city_data_df['Humidity']<35) &(city_data_df['Cloudiness']<10)&(city_data_df['Wind Speed']<5)]


#Drop any rows with null values
city_data_ideal=city_data_ideal.dropna() 

# Display sample data
city_data_ideal.head()

City_ID        City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
150      150    show low  34.2542 -110.0298     34.49        18           0   
190      190     connell  46.6635 -118.8611     30.56        24           0   
222      222   sain alto  23.5833 -103.2500     28.69        18           5   
245      245  prineville  44.2999 -120.8345     29.51        23           0   
249      249     pullman  46.7313 -117.1796     30.13        21           0   

     Wind Speed Country        Date  
150        3.60      US  1690494433  
190        1.34      US  1690494444  
222        4.79      MX  1690494453  
245        3.60      US  1690494325  
249        3.58      US  1690494458

### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_ideal[['City', 'Country', 'Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name", "")
# Display sample data
hotel_df.head()


15

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel

radius = 10000   
params = {
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel",
    "format":"json",
    "limit" : 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params) 
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
show low - nearest hotel: Holiday Inn Express Showlow
connell - nearest hotel: No hotel found
sain alto - nearest hotel: No hotel found
prineville - nearest hotel: Best Western
pullman - nearest hotel: Quality Inn
miles city - nearest hotel: Historic Olive Hotel
frontera - nearest hotel: Quirino
ellensburg - nearest hotel: Hotel Windrow
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
idri - nearest hotel: No hotel found
eilat - nearest hotel: Melony Club
khamis mushait - nearest hotel: No hotel found
scottsbluff - nearest hotel: Hampton Inn & Suites Scottsbluff Conference Center
yangi qal'ah - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
150    show low      US  34.2542 -110.0298        18   
190     connell      US  46.6635 -118.8611        24   
222   sain alto      MX  23.5833 -103.2500        18   
245  prineville      US  44.2999 -120.8345        23   
249     pullman      US  46.7313 -117.1796        21   

                      Hotel Name  
150  Holiday Inn Express Showlow  
190               No hotel found  
222               No hotel found  
245                 Best Western  
249                  Quality Inn

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display


# Configure the map plot
map2 = hotel_df.hvplot.points(
                                "Lng", 
                                "Lat", 
                                 geo = True,
                                 color = "City",
                                 alpha =1,
                                 tiles = "OSM",
                                 frame_width = 700,
                                 frame_height = 500,
                                 scale = 1,
                                 hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)